In [65]:
from dateutil import parser
import plotly.graph_objects as go
import datetime as dt
import sys
sys.path.append('../')
import pandas as pd
from api.oanda import OandaApi
from Infrastructure.instrument_collection import instrumentCollection as ic
from db.db import DataDB

In [66]:
database=DataDB()

In [67]:
ic.LoadInstrumentsDB()


In [68]:
pairs=[]
for pair,val in ic.instruments_dict.items():
    if val.ins_type=="CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [69]:
pairs

['GBP_AUD',
 'GBP_CHF',
 'GBP_CAD',
 'GBP_HKD',
 'GBP_ZAR',
 'GBP_USD',
 'GBP_SGD',
 'GBP_PLN',
 'GBP_NZD',
 'EUR_GBP',
 'GBP_JPY']

In [70]:
data=[]
api=OandaApi()
for p in pairs:
    print(p)
    d_temp= api.get_candles_df(pair_name=p,granularity='D',count=400)
    d_temp['gain']=((d_temp.mid_c-d_temp.mid_o)/d_temp.mid_c)/100
    if "_GBP" in p:
        d_temp['gain'] =d_temp['gain']*-1
    data.append(d_temp)
candles_df=pd.concat(data)
candles_df.reset_index(drop=True,inplace=True)
candles_df.time=candles_df.time.dt.date

GBP_AUD
GBP_CHF
GBP_CAD
GBP_HKD
GBP_ZAR
GBP_USD
GBP_SGD
GBP_PLN
GBP_NZD
EUR_GBP
GBP_JPY


In [76]:
candles_df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain
0,2022-12-19,316013,1.81267,1.82760,1.81158,1.82389,1.81067,1.82751,1.81016,1.82371,1.81467,1.82772,1.81174,1.82407,0.000062
1,2022-12-20,261699,1.82266,1.82728,1.80024,1.80118,1.82016,1.82714,1.80007,1.80097,1.82516,1.82796,1.80041,1.80138,-0.000119
2,2022-12-21,267077,1.80185,1.80702,1.78858,1.80500,1.79935,1.80692,1.78849,1.80483,1.80435,1.80712,1.78865,1.80518,0.000017
3,2022-12-22,248973,1.80517,1.80698,1.79136,1.79506,1.80294,1.80567,1.79121,1.79464,1.80740,1.80948,1.79149,1.79547,-0.000056
4,2022-12-26,184264,1.79229,1.79468,1.78122,1.78674,1.78979,1.79432,1.78112,1.78662,1.79479,1.79681,1.78131,1.78687,-0.000031


In [77]:
calendar_data=database.query_all(DataDB.CALENDAR_COLL)

In [78]:
calendar_data_df = pd.DataFrame.from_dict(calendar_data)

In [74]:
calendar_data_df.date=calendar_data_df.date.dt.date

In [79]:
calendar_data_df_uk=calendar_data_df[calendar_data_df.country=="united kingdom"].copy()

In [80]:
for col in ['actual',"previous","forecast"]:
    for sy in ['£','%','B','K']:
         calendar_data_df_uk[col] =calendar_data_df_uk[col].str.replace(sy,'') 
    calendar_data_df_uk[col].replace('',0,inplace=True)
    calendar_data_df_uk[col] =calendar_data_df_uk[col] .astype(float)

In [81]:
date_match={}
for orig in calendar_data_df_uk.date.unique():
    d=orig
    tries=0
    while d not in candles_df.time.values:
        d= d +dt.timedelta(days=1)
        tries+=1
        if tries > 5:
            print("Failed")
            break
    date_match[orig] = d
    

Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed


In [82]:
calendar_data_df_uk['orig_date']=calendar_data_df_uk.date
calendar_data_df_uk.date=[date_match[x] for x in calendar_data_df_uk.date]


In [86]:
calendar_data_df_uk

,date,country,category,event,symbol,actual,previous,forecast,orig_date
9,2021-05-18,united kingdom,balance of trade,balance of trade,UKTBTTBA,-2.00,-0.90,-6.10,2021-05-12
13,2021-05-24,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,-15.10,-19.40,25.00,2021-05-18
16,2021-05-25,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.50,0.70,1.40,2021-05-19
21,2021-05-26,united kingdom,consumer confidence,gfk consumer confidence,UKCCI,-9.00,-15.00,-14.00,2021-05-20
44,2021-06-17,united kingdom,balance of trade,balance of trade,UKTBTTBA,-0.90,-2.00,-3.20,2021-06-11
...,...,...,...,...,...,...,...,...,...
739,2024-03-18,united kingdom,unemployment rate,unemployment rate,UKUEILOR,3.90,3.80,3.90,2024-03-12
744,2024-03-19,united kingdom,monthly gdp mom,gdp mom,GBRMGM,0.20,-0.10,0.10,2024-03-13
755,2024-03-26,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,3.40,4.00,3.50,2024-03-20
761,2024-03-27,united kingdom,interest rate,boe interest rate decision,UKBRBASE,5.25,5.25,5.25,2024-03-21
